# Importing modules and functions

In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.Chem import rdFingerprintGenerator
from rdkit.ML.Descriptors import MoleculeDescriptors
import chembl_structure_pipeline
from molvs import standardize_smiles
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import joblib
import pickle
from numpy import savetxt
from padelpy import from_sdf
from IPython.display import HTML
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings('ignore')

[22:35:56] Initializing Normalizer


# Data entry and curation work set

In [2]:
uploaded_file_ws="datasets/KRAS_work_from_insilico.sdf"
supplier_ws = Chem.ForwardSDMolSupplier(uploaded_file_ws,sanitize=False)
failed_mols_ws = []
all_mols_ws =[]
wrong_structure_ws=[]
wrong_smiles_ws=[]
y_tr = []
y_bad_index=[]

for i, m in enumerate(supplier_ws):
    structure = Chem.Mol(m)
    all_mols_ws.append(structure)
    y_tr.append(m.GetProp("pIC50_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ws.append(m)
        wrong_smiles_ws.append(Chem.MolToSmiles(m))
        wrong_structure_ws.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ws), 'molecules')
print('Failed data: ', len(failed_mols_ws), 'molecules')
number_ws =[]
for i in range(len(failed_mols_ws)):
        number_ws.append(str(i+1))
bad_molecules_ws = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ws, 'SMILES of wrong structure: ': wrong_smiles_ws, 'No.': number_ws}, index=None)
bad_molecules_ws = bad_molecules_ws.set_index('No.')
bad_molecules_ws

Original data:  452 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [3]:
y_tr[:] = [x for i,x in enumerate(y_tr) if i not in y_bad_index]

In [4]:
len(y_tr)

452

# Standardization SDF file for work set

In [5]:
all_mols_ws[:] = [x for i,x in enumerate(all_mols_ws) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ws)):
    record = Chem.MolToSmiles(all_mols_ws[i])
    records.append(record)

moldf_ws = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ws.append(m)
    
print('Kept data: ', len(moldf_ws), 'molecules')

Kept data:  452 molecules


In [6]:
moldf_ws=pd.DataFrame(moldf_ws, columns=['Mol'])
moldf_ws

,Mol
0,<rdkit.Chem.rdchem.Mol object at 0x0000024D015...
1,<rdkit.Chem.rdchem.Mol object at 0x0000024D37F...
2,<rdkit.Chem.rdchem.Mol object at 0x0000024D37D...
3,<rdkit.Chem.rdchem.Mol object at 0x0000024D37F...
4,<rdkit.Chem.rdchem.Mol object at 0x0000024D37F...
...,...
447,<rdkit.Chem.rdchem.Mol object at 0x0000024D37F...
448,<rdkit.Chem.rdchem.Mol object at 0x0000024D37F...
449,<rdkit.Chem.rdchem.Mol object at 0x0000024D37F...
450,<rdkit.Chem.rdchem.Mol object at 0x0000024D37F...


# Data entry and curation test set

In [7]:
uploaded_file_ts="datasets/KRAS_test_from_insilico.sdf"
supplier_ts = Chem.ForwardSDMolSupplier(uploaded_file_ts,sanitize=False)
failed_mols_ts = []
all_mols_ts =[]
wrong_structure_ts=[]
wrong_smiles_ts=[]
y_ts = []
y_bad_index=[]
for i, m in enumerate(supplier_ts):
    structure = Chem.Mol(m)
    all_mols_ts.append(structure)
    y_ts.append(m.GetProp("pIC50_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ts.append(m)
        wrong_smiles_ts.append(Chem.MolToSmiles(m))
        wrong_structure_ts.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ts), 'molecules')
print('Failed data: ', len(failed_mols_ts), 'molecules')
number_ts =[]
for i in range(len(failed_mols_ts)):
        number_ts.append(str(i+1))
bad_molecules_ts = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ts, 'SMILES of wrong structure: ': wrong_smiles_ts, 'No.': number_ts}, index=None)
bad_molecules_ts = bad_molecules_ts.set_index('No.')
bad_molecules_ts

Original data:  114 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [8]:
y_ts[:] = [x for i,x in enumerate(y_ts) if i not in y_bad_index]

In [9]:
len(y_ts)

114

# Standardization SDF file for test set

In [10]:
all_mols_ts[:] = [x for i,x in enumerate(all_mols_ts) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ts)):
    record = Chem.MolToSmiles(all_mols_ts[i])
    records.append(record)

moldf_ts = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ts.append(m)
    
print('Kept data: ', len(moldf_ts), 'molecules')

Kept data:  114 molecules


In [11]:
moldf_ts=pd.DataFrame(moldf_ts, columns=['Mol'])
moldf_ts

,Mol
0,<rdkit.Chem.rdchem.Mol object at 0x0000024D37F...
1,<rdkit.Chem.rdchem.Mol object at 0x0000024D37F...
2,<rdkit.Chem.rdchem.Mol object at 0x0000024D37F...
3,<rdkit.Chem.rdchem.Mol object at 0x0000024D37F...
4,<rdkit.Chem.rdchem.Mol object at 0x0000024D37F...
...,...
109,<rdkit.Chem.rdchem.Mol object at 0x0000024D37F...
110,<rdkit.Chem.rdchem.Mol object at 0x0000024D37F...
111,<rdkit.Chem.rdchem.Mol object at 0x0000024D37F...
112,<rdkit.Chem.rdchem.Mol object at 0x0000024D37F...


# Calculation Topological Torsion Fingerprints for work set

In [12]:
# import warnings
# from rdkit import RDLogger

# # Suppress RDKit warnings (including deprecation warnings)
# RDLogger.DisableLog('rdApp.*')

# def calcfp(mol, funcFPInfo=dict(radius=2, nBits=1024, useFeatures=False, useChirality=False)):
#     fp = AllChem.GetMorganFingerprintAsBitVect(mol, **funcFPInfo)
#     fp = pd.Series(np.asarray(fp))
#     fp = fp.add_prefix('Bit_')
#     return fp

In [13]:
import warnings
from rdkit import RDLogger
import numpy as np
import pandas as pd
from rdkit.Chem import AllChem

# Suppress RDKit warnings
RDLogger.DisableLog('rdApp.*')

def calcfp(mol, funcFPInfo=dict(size=1024)):
    """
    Calculate Topological Torsion fingerprint (RDKit)
    Returns: Pandas Series with 'Bit_*' features
    """

    # Create TT generator with desired size
    fpgen = AllChem.GetTopologicalTorsionGenerator(fpSize=funcFPInfo.get("size", 1024))

    # Calculate fingerprint (IntSparseIntVect → dense bitvector)
    fp = fpgen.GetFingerprint(mol)

    # Convert to numpy array
    arr = np.asarray(fp)

    # Convert to Pandas Series
    fp = pd.Series(arr)
    fp = fp.add_prefix("Bit_")

    return fp


In [14]:
# Training set
desc_ws = moldf_ws.Mol.apply(calcfp)
desc_ws

,Bit_0,Bit_1,Bit_2,Bit_3,Bit_4,Bit_5,Bit_6,Bit_7,Bit_8,Bit_9,...,Bit_1014,Bit_1015,Bit_1016,Bit_1017,Bit_1018,Bit_1019,Bit_1020,Bit_1021,Bit_1022,Bit_1023
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
448,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
449,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
450,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
savetxt('models/Topological_Torsion/x_tr_MF.csv', desc_ws, delimiter=',')

In [16]:
y_tr = np.array(y_tr, dtype=np.float32)
len(y_tr)

452

# Calculation Topological Torsion Fingerprints for test set

In [17]:
desc_ts = moldf_ts.Mol.apply(calcfp)
desc_ts

,Bit_0,Bit_1,Bit_2,Bit_3,Bit_4,Bit_5,Bit_6,Bit_7,Bit_8,Bit_9,...,Bit_1014,Bit_1015,Bit_1016,Bit_1017,Bit_1018,Bit_1019,Bit_1020,Bit_1021,Bit_1022,Bit_1023
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
110,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
111,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
112,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
y_ts = np.array(y_ts, dtype=np.float32)
len(y_ts)

114

# BASELINE

 ## GradientBoostingRegressor model building and validation

In [19]:
seed = 42

In [20]:
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [21]:
%%time
model = CatBoostRegressor()
parameters = {'depth' : [6,8,10],
              'learning_rate' : [0.01, 0.05, 0.1],
              'iterations'    : [100,500, 1000]
              }

grid = GridSearchCV(estimator=model, param_grid = parameters, n_jobs=-1, cv = cv)
grid.fit(desc_ws, y_tr, verbose=False)

CPU times: total: 1min
Wall time: 12min 27s


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=<catboost.core.CatBoostRegressor object at 0x0000024D3A4E9E20>,
             n_jobs=-1,
             param_grid={'depth': [6, 8, 10], 'iterations': [100, 500, 1000],
                         'learning_rate': [0.01, 0.05, 0.1]})

In [22]:
best_CatBR = grid.best_estimator_

In [23]:
grid.best_params_

{'depth': 6, 'iterations': 500, 'learning_rate': 0.05}

In [24]:
params={'verbose': False}

In [25]:
%%time
y_pred_CV_CatBR = cross_val_predict(best_CatBR, desc_ws, y_tr, cv=cv, params=params)

CPU times: total: 2min 30s
Wall time: 11.2 s


In [26]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_CatBR), 2)
Q2_CV

0.65

In [27]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_CatBR)), 2)
RMSE_CV

0.73

In [28]:
y_pred_GBR = best_CatBR.predict(desc_ts)

In [31]:
Q2_TS = round(r2_score(y_ts, y_pred_GBR), 2)
Q2_TS

0.67

In [32]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_GBR)), 2)
RMSE_TS

0.71

save the model to disk

In [33]:
pickle.dump(best_CatBR, open('Models/Topological_Torsion/CatBoost_Topological_Torsion.pkl', 'wb'))

load the model from disk

In [123]:
best_CatBR = pickle.load(open('Models/Topological_Torsion/CatBoost_Topological_Torsion.pkl', 'rb'))

# Estimating applicability domain. Method - Euclidian distances, K=1

In [34]:
neighbors_k= pairwise_distances(desc_ws, n_jobs=-1)
neighbors_k.sort(0)

In [35]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,442,443,444,445,446,447,448,449,450,451
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,4.000000,3.872983,1.732051,2.449490,1.000000,3.316625,2.449490,4.000000,2.645751,4.690416,...,2.645751,2.645751,2.645751,2.236068,2.000000,2.449490,3.162278,2.449490,2.645751,3.605551
2,4.123106,3.872983,2.000000,3.605551,1.732051,3.741657,3.162278,4.582576,4.358899,5.744563,...,2.828427,2.645751,2.828427,3.162278,2.645751,2.828427,4.123106,2.828427,2.828427,5.291503
3,4.358899,4.000000,2.000000,3.741657,3.162278,3.741657,3.162278,4.690416,4.582576,5.916080,...,2.828427,2.828427,3.000000,3.464102,2.645751,2.828427,4.123106,3.162278,2.828427,5.477226
4,4.472136,4.000000,2.000000,3.872983,3.316625,3.872983,3.316625,4.690416,4.582576,6.000000,...,3.162278,2.828427,3.162278,3.605551,2.645751,2.828427,4.582576,3.316625,3.000000,5.477226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,9.433981,9.591663,9.055385,9.165151,9.165151,9.539392,9.055385,9.110434,9.848858,8.831761,...,9.219544,9.327379,9.110434,9.433981,9.110434,9.219544,9.433981,9.433981,9.165151,10.246951
448,9.433981,9.643651,9.219544,9.327379,9.165151,9.695360,9.110434,9.219544,9.848858,9.000000,...,9.273618,9.486833,9.110434,9.433981,9.165151,9.327379,9.433981,9.539392,9.165151,10.295630
449,9.486833,9.797959,9.433981,9.643651,9.273618,9.899495,9.327379,9.273618,9.899495,9.000000,...,9.380832,9.539392,9.165151,9.539392,9.165151,9.327379,9.486833,9.643651,9.165151,10.440307
450,9.746794,10.295630,9.949874,9.949874,9.433981,10.295630,9.591663,9.695360,10.198039,9.165151,...,9.643651,9.539392,9.539392,9.695360,9.273618,9.486833,9.949874,9.797959,9.539392,10.488088


In [36]:
similarity= neighbors_k

In [37]:
Dmean=np.mean(similarity[1,:])

In [38]:
round(Dmean, 2)

2.84

In [39]:
std=np.std(similarity[1,:])

In [40]:
round(std, 2)

0.92

In [41]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

3.3


In [42]:
neighbors_k_ts= pairwise_distances(desc_ws,Y=desc_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [43]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,1.732051,2.000000,3.316625,3.316625,3.316625,5.000000,2.000000,2.000000,1.000000,1.732051,...,3.162278,3.316625,2.449490,2.645751,2.000000,3.316625,2.236068,3.000000,2.645751,3.000000
1,4.795832,2.449490,3.316625,3.872983,3.464102,5.099020,2.236068,2.236068,1.000000,2.000000,...,3.741657,4.690416,2.645751,3.000000,2.236068,3.464102,2.236068,3.162278,3.000000,3.162278
2,4.795832,2.645751,3.464102,4.000000,3.605551,5.196152,2.449490,2.449490,1.732051,2.236068,...,4.472136,4.795832,3.000000,3.000000,2.236068,3.464102,2.236068,4.000000,3.000000,3.162278
3,4.795832,2.645751,3.605551,4.123106,3.605551,5.196152,2.449490,2.449490,2.000000,2.449490,...,5.099020,5.385165,3.162278,3.000000,2.645751,3.741657,2.236068,4.472136,3.000000,3.316625
4,4.898979,2.828427,3.741657,4.123106,3.605551,5.291503,2.449490,2.449490,2.449490,2.449490,...,5.385165,5.477226,3.316625,3.162278,2.645751,3.741657,2.449490,4.690416,3.162278,3.316625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,9.643651,9.110434,9.433981,8.888194,9.433981,9.848858,8.888194,9.165151,9.219544,8.944272,...,9.899495,9.643651,9.110434,9.165151,9.055385,9.380832,9.055385,9.591663,9.055385,9.273618
448,9.643651,9.165151,9.433981,9.000000,9.695360,10.000000,9.055385,9.327379,9.219544,9.110434,...,9.949874,9.695360,9.165151,9.165151,9.055385,9.380832,9.273618,9.643651,9.110434,9.327379
449,9.695360,9.486833,9.433981,9.327379,9.695360,10.099505,9.055385,9.327379,9.486833,9.110434,...,9.949874,9.746794,9.273618,9.380832,9.110434,9.380832,9.273618,9.746794,9.327379,9.327379
450,10.099505,10.000000,9.591663,9.380832,10.198039,10.488088,9.273618,9.848858,9.746794,9.327379,...,9.949874,10.000000,9.327379,9.539392,9.486833,9.746794,9.433981,9.848858,9.486833,9.539392


In [44]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[1.732 2.    3.317 3.317 3.317 5.    2.    2.    1.    1.732 4.69  4.
 3.464 3.742 3.162 3.464 4.359 3.873 3.162 2.449 5.    2.449 3.742 2.828
 2.828 3.    5.    2.646 2.828 2.    0.    2.236 2.236 2.646 3.464 2.449
 1.414 1.732 5.477 2.236 1.    3.317 2.828 2.449 2.449 1.414 1.414 2.
 2.449 2.828 2.828 3.    2.828 1.414 3.    2.828 3.464 3.    4.583 4.
 1.732 3.742 2.646 2.449 2.236 2.236 3.162 3.317 2.828 1.414 2.449 3.162
 3.162 2.646 1.414 3.    2.449 2.    1.732 4.123 3.606 5.196 1.414 2.449
 3.    2.646 2.646 2.828 4.    2.449 2.828 2.449 3.    2.646 2.    2.449
 2.828 2.    2.    1.    2.646 2.236 3.162 2.646 3.162 3.317 2.449 2.646
 2.    3.317 2.236 3.    2.646 3.   ]


In [45]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True False False False False  True  True  True  True False False
 False False  True False False False  True  True False  True False  True
  True  True False  True  True  True  True  True  True  True False  True
  True  True False  True  True False  True  True  True  True  True  True
  True  True  True  True  True  True  True  True False  True False False
  True False  True  True  True  True  True False  True  True  True  True
  True  True  True  True  True  True  True False False False  True  True
  True  True  True  True False  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True False  True  True
  True False  True  True  True  True]


In [46]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD),2))

Coverage =  0.75


In [47]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  0   1   6   7   8   9  14  18  19  21  23  24  25  27  28  29  30  31
  32  33  35  36  37  39  40  42  43  44  45  46  47  48  49  50  51  52
  53  54  55  57  60  62  63  64  65  66  68  69  70  71  72  73  74  75
  76  77  78  82  83  84  85  86  87  89  90  91  92  93  94  95  96  97
  98  99 100 101 102 103 104 106 107 108 110 111 112 113]


In [48]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# Prediction only for molecules included in  AD

In [49]:
y_pred_GBR_ad=list(y_pred_GBR)

In [50]:
y_pred_GBR_ad[:] = [x for i,x in enumerate(y_pred_GBR_ad) if i not in out_Ad]

In [51]:
len(y_pred_GBR_ad)

86

In [52]:
y_ts_ad=list(y_ts)

In [53]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [54]:
len(y_ts_ad)

86

In [55]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_GBR_ad), 2)
Q2_TS

0.61

In [56]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_GBR_ad)), 2)
RMSE_TS

0.74

# SVM model building and validation

In [57]:
from sklearn.svm import SVR

In [58]:
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}

In [59]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [60]:
svm = GridSearchCV(SVR(C=1.0, epsilon=0.2), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [61]:
svm.fit(desc_ws, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVR(epsilon=0.2), n_jobs=-1,
             param_grid={'C': [1, 10, 100, 1000, 10000],
                         'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]},
             verbose=1)

In [62]:
svm.best_params_
best_svm = svm.best_estimator_

In [63]:
svm.best_params_

{'C': 10, 'gamma': 0.01}

In [64]:
y_pred_CV_svm = cross_val_predict(best_svm, desc_ws, y_tr, cv=cv)

In [65]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_svm), 2)
Q2_CV

0.66

In [66]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_svm)), 2)
RMSE_CV

0.71

#  Prediction for test set's molecules

In [67]:
x_ts = np.array(desc_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [68]:
y_pred_svm = best_svm.predict(x_ts)

In [69]:
Q2_TS = round(r2_score(y_ts, y_pred_svm), 2)
Q2_TS

0.65

In [70]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_svm)), 2)
RMSE_TS

0.73

save the model to disk

In [71]:
pickle.dump(best_svm, open('Models/Topological_Torsion/Morgan_Topological_Torsion.pkl', 'wb'))

load the model from disk

In [165]:
best_svm = pickle.load(open('Models/Topological_Torsion/Morgan_Topological_Torsion.pkl.pkl', 'rb'))

# Estimating applicability domain. Method - Euclidian distances, K=1

In [72]:
neighbors_k= pairwise_distances(desc_ws, n_jobs=-1)
neighbors_k.sort(0)

In [73]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,442,443,444,445,446,447,448,449,450,451
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,4.000000,3.872983,1.732051,2.449490,1.000000,3.316625,2.449490,4.000000,2.645751,4.690416,...,2.645751,2.645751,2.645751,2.236068,2.000000,2.449490,3.162278,2.449490,2.645751,3.605551
2,4.123106,3.872983,2.000000,3.605551,1.732051,3.741657,3.162278,4.582576,4.358899,5.744563,...,2.828427,2.645751,2.828427,3.162278,2.645751,2.828427,4.123106,2.828427,2.828427,5.291503
3,4.358899,4.000000,2.000000,3.741657,3.162278,3.741657,3.162278,4.690416,4.582576,5.916080,...,2.828427,2.828427,3.000000,3.464102,2.645751,2.828427,4.123106,3.162278,2.828427,5.477226
4,4.472136,4.000000,2.000000,3.872983,3.316625,3.872983,3.316625,4.690416,4.582576,6.000000,...,3.162278,2.828427,3.162278,3.605551,2.645751,2.828427,4.582576,3.316625,3.000000,5.477226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,9.433981,9.591663,9.055385,9.165151,9.165151,9.539392,9.055385,9.110434,9.848858,8.831761,...,9.219544,9.327379,9.110434,9.433981,9.110434,9.219544,9.433981,9.433981,9.165151,10.246951
448,9.433981,9.643651,9.219544,9.327379,9.165151,9.695360,9.110434,9.219544,9.848858,9.000000,...,9.273618,9.486833,9.110434,9.433981,9.165151,9.327379,9.433981,9.539392,9.165151,10.295630
449,9.486833,9.797959,9.433981,9.643651,9.273618,9.899495,9.327379,9.273618,9.899495,9.000000,...,9.380832,9.539392,9.165151,9.539392,9.165151,9.327379,9.486833,9.643651,9.165151,10.440307
450,9.746794,10.295630,9.949874,9.949874,9.433981,10.295630,9.591663,9.695360,10.198039,9.165151,...,9.643651,9.539392,9.539392,9.695360,9.273618,9.486833,9.949874,9.797959,9.539392,10.488088


In [74]:
similarity= neighbors_k

In [75]:
Dmean=np.mean(similarity[1,:])

In [76]:
round(Dmean, 2)

2.84

In [77]:
std=np.std(similarity[1,:])

In [78]:
round(std, 2)

0.92

In [79]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

3.3


In [80]:
neighbors_k_ts= pairwise_distances(desc_ws,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [81]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,1.732051,2.000000,3.316625,3.316625,3.316625,5.000000,2.000000,2.000000,1.000000,1.732051,...,3.162278,3.316625,2.449490,2.645751,2.000000,3.316625,2.236068,3.000000,2.645751,3.000000
1,4.795832,2.449490,3.316625,3.872983,3.464102,5.099020,2.236068,2.236068,1.000000,2.000000,...,3.741657,4.690416,2.645751,3.000000,2.236068,3.464102,2.236068,3.162278,3.000000,3.162278
2,4.795832,2.645751,3.464102,4.000000,3.605551,5.196152,2.449490,2.449490,1.732051,2.236068,...,4.472136,4.795832,3.000000,3.000000,2.236068,3.464102,2.236068,4.000000,3.000000,3.162278
3,4.795832,2.645751,3.605551,4.123106,3.605551,5.196152,2.449490,2.449490,2.000000,2.449490,...,5.099020,5.385165,3.162278,3.000000,2.645751,3.741657,2.236068,4.472136,3.000000,3.316625
4,4.898979,2.828427,3.741657,4.123106,3.605551,5.291503,2.449490,2.449490,2.449490,2.449490,...,5.385165,5.477226,3.316625,3.162278,2.645751,3.741657,2.449490,4.690416,3.162278,3.316625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,9.643651,9.110434,9.433981,8.888194,9.433981,9.848858,8.888194,9.165151,9.219544,8.944272,...,9.899495,9.643651,9.110434,9.165151,9.055385,9.380832,9.055385,9.591663,9.055385,9.273618
448,9.643651,9.165151,9.433981,9.000000,9.695360,10.000000,9.055385,9.327379,9.219544,9.110434,...,9.949874,9.695360,9.165151,9.165151,9.055385,9.380832,9.273618,9.643651,9.110434,9.327379
449,9.695360,9.486833,9.433981,9.327379,9.695360,10.099505,9.055385,9.327379,9.486833,9.110434,...,9.949874,9.746794,9.273618,9.380832,9.110434,9.380832,9.273618,9.746794,9.327379,9.327379
450,10.099505,10.000000,9.591663,9.380832,10.198039,10.488088,9.273618,9.848858,9.746794,9.327379,...,9.949874,10.000000,9.327379,9.539392,9.486833,9.746794,9.433981,9.848858,9.486833,9.539392


In [82]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[1.732 2.    3.317 3.317 3.317 5.    2.    2.    1.    1.732 4.69  4.
 3.464 3.742 3.162 3.464 4.359 3.873 3.162 2.449 5.    2.449 3.742 2.828
 2.828 3.    5.    2.646 2.828 2.    0.    2.236 2.236 2.646 3.464 2.449
 1.414 1.732 5.477 2.236 1.    3.317 2.828 2.449 2.449 1.414 1.414 2.
 2.449 2.828 2.828 3.    2.828 1.414 3.    2.828 3.464 3.    4.583 4.
 1.732 3.742 2.646 2.449 2.236 2.236 3.162 3.317 2.828 1.414 2.449 3.162
 3.162 2.646 1.414 3.    2.449 2.    1.732 4.123 3.606 5.196 1.414 2.449
 3.    2.646 2.646 2.828 4.    2.449 2.828 2.449 3.    2.646 2.    2.449
 2.828 2.    2.    1.    2.646 2.236 3.162 2.646 3.162 3.317 2.449 2.646
 2.    3.317 2.236 3.    2.646 3.   ]


In [83]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True False False False False  True  True  True  True False False
 False False  True False False False  True  True False  True False  True
  True  True False  True  True  True  True  True  True  True False  True
  True  True False  True  True False  True  True  True  True  True  True
  True  True  True  True  True  True  True  True False  True False False
  True False  True  True  True  True  True False  True  True  True  True
  True  True  True  True  True  True  True False False False  True  True
  True  True  True  True False  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True False  True  True
  True False  True  True  True  True]


In [84]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.75


In [85]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  0   1   6   7   8   9  14  18  19  21  23  24  25  27  28  29  30  31
  32  33  35  36  37  39  40  42  43  44  45  46  47  48  49  50  51  52
  53  54  55  57  60  62  63  64  65  66  68  69  70  71  72  73  74  75
  76  77  78  82  83  84  85  86  87  89  90  91  92  93  94  95  96  97
  98  99 100 101 102 103 104 106 107 108 110 111 112 113]


In [86]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# Prediction only for molecules included in  AD

In [87]:
y_pred_svm_ad=list(y_pred_svm)

In [88]:
y_pred_svm_ad[:] = [x for i,x in enumerate(y_pred_svm_ad) if i not in out_Ad]

In [89]:
len(y_pred_svm_ad)

86

In [90]:
y_ts_ad=list(y_ts)

In [91]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [92]:
len(y_ts_ad)

86

In [93]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_svm_ad), 2)
Q2_TS

0.6

In [94]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_svm_ad)), 2)
RMSE_TS

0.74

# Multi-layer Perceptron regressor

In [95]:
from sklearn.neural_network import MLPRegressor

In [96]:
param_grid ={"hidden_layer_sizes": [(400, 300, 200, 100),(100, 100, 100), (10, 10, 10),(50,)], "activation": ["tanh", "relu"], "solver": ["lbfgs", "sgd", "adam"], "alpha": [0.00005,0.0005], 'max_iter': [1000, 2000]}

In [97]:
m = GridSearchCV(MLPRegressor(), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [98]:
m.fit(desc_ws, y_tr)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=MLPRegressor(), n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'],
                         'alpha': [5e-05, 0.0005],
                         'hidden_layer_sizes': [(400, 300, 200, 100),
                                                (100, 100, 100), (10, 10, 10),
                                                (50,)],
                         'max_iter': [1000, 2000],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             verbose=1)

In [99]:
best_MLPR = m.best_estimator_

In [100]:
y_pred_CV_MLPR = cross_val_predict(best_MLPR, desc_ws, y_tr, cv=cv)

In [101]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_MLPR), 2)
Q2_CV

0.63

In [102]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_MLPR)), 2)
RMSE_CV

0.74

# Prediction for test set's molecules

In [103]:
y_pred_MLPR = best_MLPR.predict(desc_ts)

In [104]:
Q2_TS = round(r2_score(y_ts, y_pred_MLPR), 2)
Q2_TS

0.6

In [105]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_MLPR)), 2)
RMSE_TS

0.79

save the model to disk

In [106]:
pickle.dump(best_MLPR, open('models/Topological_Torsion/MLPR_Topological_Torsion.pkl', 'wb'))

#  Estimating applicability domain. Method - Euclidian distances, K=1

In [107]:
neighbors_k= pairwise_distances(desc_ws, n_jobs=-1)
neighbors_k.sort(0)

In [108]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,442,443,444,445,446,447,448,449,450,451
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,4.000000,3.872983,1.732051,2.449490,1.000000,3.316625,2.449490,4.000000,2.645751,4.690416,...,2.645751,2.645751,2.645751,2.236068,2.000000,2.449490,3.162278,2.449490,2.645751,3.605551
2,4.123106,3.872983,2.000000,3.605551,1.732051,3.741657,3.162278,4.582576,4.358899,5.744563,...,2.828427,2.645751,2.828427,3.162278,2.645751,2.828427,4.123106,2.828427,2.828427,5.291503
3,4.358899,4.000000,2.000000,3.741657,3.162278,3.741657,3.162278,4.690416,4.582576,5.916080,...,2.828427,2.828427,3.000000,3.464102,2.645751,2.828427,4.123106,3.162278,2.828427,5.477226
4,4.472136,4.000000,2.000000,3.872983,3.316625,3.872983,3.316625,4.690416,4.582576,6.000000,...,3.162278,2.828427,3.162278,3.605551,2.645751,2.828427,4.582576,3.316625,3.000000,5.477226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,9.433981,9.591663,9.055385,9.165151,9.165151,9.539392,9.055385,9.110434,9.848858,8.831761,...,9.219544,9.327379,9.110434,9.433981,9.110434,9.219544,9.433981,9.433981,9.165151,10.246951
448,9.433981,9.643651,9.219544,9.327379,9.165151,9.695360,9.110434,9.219544,9.848858,9.000000,...,9.273618,9.486833,9.110434,9.433981,9.165151,9.327379,9.433981,9.539392,9.165151,10.295630
449,9.486833,9.797959,9.433981,9.643651,9.273618,9.899495,9.327379,9.273618,9.899495,9.000000,...,9.380832,9.539392,9.165151,9.539392,9.165151,9.327379,9.486833,9.643651,9.165151,10.440307
450,9.746794,10.295630,9.949874,9.949874,9.433981,10.295630,9.591663,9.695360,10.198039,9.165151,...,9.643651,9.539392,9.539392,9.695360,9.273618,9.486833,9.949874,9.797959,9.539392,10.488088


In [109]:
similarity= neighbors_k

In [110]:
Dmean=np.mean(similarity[1,:])

In [111]:
round(Dmean, 2)

2.84

In [112]:
std=np.std(similarity[1,:])

In [113]:
round(std, 2)

0.92

In [114]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

3.3


In [115]:
neighbors_k_ts= pairwise_distances(desc_ws,Y=desc_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [116]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,1.732051,2.000000,3.316625,3.316625,3.316625,5.000000,2.000000,2.000000,1.000000,1.732051,...,3.162278,3.316625,2.449490,2.645751,2.000000,3.316625,2.236068,3.000000,2.645751,3.000000
1,4.795832,2.449490,3.316625,3.872983,3.464102,5.099020,2.236068,2.236068,1.000000,2.000000,...,3.741657,4.690416,2.645751,3.000000,2.236068,3.464102,2.236068,3.162278,3.000000,3.162278
2,4.795832,2.645751,3.464102,4.000000,3.605551,5.196152,2.449490,2.449490,1.732051,2.236068,...,4.472136,4.795832,3.000000,3.000000,2.236068,3.464102,2.236068,4.000000,3.000000,3.162278
3,4.795832,2.645751,3.605551,4.123106,3.605551,5.196152,2.449490,2.449490,2.000000,2.449490,...,5.099020,5.385165,3.162278,3.000000,2.645751,3.741657,2.236068,4.472136,3.000000,3.316625
4,4.898979,2.828427,3.741657,4.123106,3.605551,5.291503,2.449490,2.449490,2.449490,2.449490,...,5.385165,5.477226,3.316625,3.162278,2.645751,3.741657,2.449490,4.690416,3.162278,3.316625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,9.643651,9.110434,9.433981,8.888194,9.433981,9.848858,8.888194,9.165151,9.219544,8.944272,...,9.899495,9.643651,9.110434,9.165151,9.055385,9.380832,9.055385,9.591663,9.055385,9.273618
448,9.643651,9.165151,9.433981,9.000000,9.695360,10.000000,9.055385,9.327379,9.219544,9.110434,...,9.949874,9.695360,9.165151,9.165151,9.055385,9.380832,9.273618,9.643651,9.110434,9.327379
449,9.695360,9.486833,9.433981,9.327379,9.695360,10.099505,9.055385,9.327379,9.486833,9.110434,...,9.949874,9.746794,9.273618,9.380832,9.110434,9.380832,9.273618,9.746794,9.327379,9.327379
450,10.099505,10.000000,9.591663,9.380832,10.198039,10.488088,9.273618,9.848858,9.746794,9.327379,...,9.949874,10.000000,9.327379,9.539392,9.486833,9.746794,9.433981,9.848858,9.486833,9.539392


In [117]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[1.732 2.    3.317 3.317 3.317 5.    2.    2.    1.    1.732 4.69  4.
 3.464 3.742 3.162 3.464 4.359 3.873 3.162 2.449 5.    2.449 3.742 2.828
 2.828 3.    5.    2.646 2.828 2.    0.    2.236 2.236 2.646 3.464 2.449
 1.414 1.732 5.477 2.236 1.    3.317 2.828 2.449 2.449 1.414 1.414 2.
 2.449 2.828 2.828 3.    2.828 1.414 3.    2.828 3.464 3.    4.583 4.
 1.732 3.742 2.646 2.449 2.236 2.236 3.162 3.317 2.828 1.414 2.449 3.162
 3.162 2.646 1.414 3.    2.449 2.    1.732 4.123 3.606 5.196 1.414 2.449
 3.    2.646 2.646 2.828 4.    2.449 2.828 2.449 3.    2.646 2.    2.449
 2.828 2.    2.    1.    2.646 2.236 3.162 2.646 3.162 3.317 2.449 2.646
 2.    3.317 2.236 3.    2.646 3.   ]


In [118]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True False False False False  True  True  True  True False False
 False False  True False False False  True  True False  True False  True
  True  True False  True  True  True  True  True  True  True False  True
  True  True False  True  True False  True  True  True  True  True  True
  True  True  True  True  True  True  True  True False  True False False
  True False  True  True  True  True  True False  True  True  True  True
  True  True  True  True  True  True  True False False False  True  True
  True  True  True  True False  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True False  True  True
  True False  True  True  True  True]


In [119]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.75


In [120]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  0   1   6   7   8   9  14  18  19  21  23  24  25  27  28  29  30  31
  32  33  35  36  37  39  40  42  43  44  45  46  47  48  49  50  51  52
  53  54  55  57  60  62  63  64  65  66  68  69  70  71  72  73  74  75
  76  77  78  82  83  84  85  86  87  89  90  91  92  93  94  95  96  97
  98  99 100 101 102 103 104 106 107 108 110 111 112 113]


In [121]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# Prediction only for molecules included in  AD

In [122]:
y_pred_MLPR_ad=list(y_pred_MLPR)

In [123]:
y_pred_MLPR_ad[:] = [x for i,x in enumerate(y_pred_MLPR_ad) if i not in out_Ad]

In [124]:
len(y_pred_MLPR_ad)

86

In [125]:
y_ts_ad=list(y_ts)

In [126]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [127]:
len(y_ts_ad)

86

In [128]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_MLPR_ad), 2)
Q2_TS

0.55

In [129]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_MLPR_ad)), 2)
RMSE_TS

0.79